In [ ]:
%pip install numpy pandas tzdata
import pyodide_js
await pyodide_js.loadPackage("https://storage.googleapis.com/ibis-wasm-wheels/pyarrow-16.0.0.dev2661%2Bg9bddb87fd-cp311-cp311-emscripten_3_1_46_wasm32.whl")
import pyarrow as pa,pandas as pd
import kuzu_wasm
from kuzu_wasm.utils import *

In [ ]:
kuzu = await kuzu_wasm.init()
db = await kuzu.Database()
conn = await kuzu.Connection(db)

In [ ]:
conn.execute("CREATE NODE TABLE User(name STRING, age INT64, PRIMARY KEY (name))")
conn.execute("CREATE NODE TABLE City(name STRING, population INT64, PRIMARY KEY (name))")
conn.execute("CREATE REL TABLE Follows(FROM User TO User, since INT64)")
conn.execute("CREATE REL TABLE LivesIn(FROM User TO City)")
conn.execute(f'COPY User FROM "{kuzu.upload("user.csv")}";')
conn.execute(f'COPY City FROM "{kuzu.upload("city.csv")}";')
conn.execute(f'COPY Follows FROM "{kuzu.upload("follows.csv")}";')
conn.execute(f'COPY LivesIn FROM "{kuzu.upload("lives_in.csv")}";')

In [ ]:
res = await conn.execute("MATCH (a:User)-[f:Follows]->(b:User)RETURN a.name, b.name, f.since")
df = toDf(res)
df